In [1]:
import pandas as pd
import os
import pickle

In [2]:
#Download the initial test statistics
df = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
#df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)


In [38]:
os.getcwd()

'c:\\Users\\jenna\\Documents\\Imperial\\Project_2\\Code'

In [3]:
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
#path = os.getcwd() + '\\Data\\permutation_test_files_proteomics'

In [37]:
#Code to read all files that start with 'Run' in a folder

final_list = []

for filename in os.listdir(path): #also lists directories
    print(filename)
    if filename.startswith('Run'):
        file_num = int(filename[3:-4])  #obtain the number of the Run
        if file_num < 11:
            print(filename)
            with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
                list1 = pickle.load(f)
            final_list.append(list1)

initial_tstats.csv
Results1
Results2
Run1.txt
Run1.txt
Run10.txt
Run10.txt
Run2.txt
Run2.txt
Run3.txt
Run3.txt
Run4.txt
Run4.txt
Run5.txt
Run5.txt
Run6.txt
Run6.txt
Run7.txt
Run7.txt
Run8.txt
Run8.txt
Run9.txt
Run9.txt
sig_edges_100kpermutations_pval0.01.txt


In [23]:
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
print(os.getcwd())

c:\Users\jenna\Documents\Imperial\Project_2\Code


In [35]:
#Code to read all files in folder that start with 'Results'
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
final_list = []

for filename in os.listdir(path): #also lists directories
    if filename.startswith('Results'):
        print(filename)
        #for file in os.listdir(path + '\\' + filename): 
        for file in os.listdir(os.path.join(path,filename)):
            print(file)
            with open(os.path.join(path, filename,file), 'rb') as f: # open in readonly mode
                list1 = pickle.load(f)
            final_list.append(list1)

Results1
Run11.txt
Run12.txt
Results2
Run13.txt
Run14.txt


In [42]:
#In a folder with directories list (Results1, Results2, Results3 etc) go through each as a separate array job

#index_num = sys.argv[1]  #this should return the array number within the array job
index_num = 1 #for example
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics\\Results' + str(index_num)
final_list = []

for filename in os.listdir(path): #also lists directories
    print(filename)
    with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
        list1 = pickle.load(f)
    final_list.append(list1)

Run11.txt
Run12.txt


In [43]:
len(final_list)

for list1 in final_list:
    print(len(list1))
    print(list1[:1])

10296
[0.24337239352125428]
10296
[-0.20153733476360788]


Calculates number of permuted values above the initial test statistic, then divides by number of tests and works out p-value, before outputting the significant edges

In [ ]:
sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   
            counter += 1
    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

Calculates number of permuted values above the initial test statistic (more scalable)

In [ ]:
num_vals = []  #number of permuted values above the magnitude of the test statistic
#sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   
            counter += 1
    num_vals.append(counter)
        
#print(len(sig_edges))
num_vals

### Another alternative: (but converting to numpy array actually seems to take up more memory)

In [ ]:
import numpy as np

final_matrix = np.empty((0,len(df)), int)

for filename in os.listdir(path):
    if filename != 'initial_tstats.csv':
        with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
            list1 = pickle.load(f)
        final_matrix = np.append(final_matrix, np.array([list1]), axis=0)

In [ ]:
final_matrix.shape

In [ ]:
final_matrix = np.array(final_list) 

final_matrix.shape

sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    col = abs(final_matrix[:,index])  #index a column and turn into absolute values
    counter = (col >= comparison).sum()

    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

In [ ]:
#comparing the size of the object from the first method vs second method
from sys import getsizeof
print(getsizeof(final_list))
print(getsizeof(final_matrix))

In [ ]:
with open ('metabolomics/Data/sig_edges.txt', 'w') as file:
     file.write(','.join(str(i) for i in sig_edges))

### Read in significant edges files and compare the results

In [ ]:
with open('Data/permutation_test_files_metabolomics/sig_edges_100kpermutations.txt') as f:
    lines = f.readlines()

sig_list = lines[0].split(';')
print(len(sig_list))
print(sig_list[0])


#252
#282
#268
#258 out of 10,296 edges significant at p < 0.01
